In [ ]:
#Packages
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
#Get Data
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
#Get all offspeed
os = all_pitches[all_pitches['TaggedPitchType'].isin(['ChangeUp', 'Splitter'])]
print(os.shape)

In [ ]:
#Subset offspeed into platoon splits
os_rr = os[(os['PitcherThrows'] == 'Right') & (os['BatterSide'] == 'Right')]
os_rl = os[(os['PitcherThrows'] == 'Right') & (os['BatterSide'] == 'Left')]
os_lr = os[(os['PitcherThrows'] == 'Left') & (os['BatterSide'] == 'Right')]
os_ll = os[(os['PitcherThrows'] == 'Left') & (os['BatterSide'] == 'Left')]

print(os_rr.shape)
print(os_rl.shape)
print(os_lr.shape)
print(os_ll.shape)

In [ ]:
#Whiffs
os_rr['whiff'] = np.where(os_rr['PitchCall']=='StrikeSwinging',1,0)
os_rl['whiff'] = np.where(os_rl['PitchCall']=='StrikeSwinging',1,0)
os_lr['whiff'] = np.where(os_lr['PitchCall']=='StrikeSwinging',1,0)
os_ll['whiff'] = np.where(os_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

Righty vs. Righty Split

In [ ]:
os_rr_X = os_rr[preds]
os_rr_y = os_rr['whiff']

In [ ]:
#Train Model: RHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(os_rr_X, os_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Auburn Subset
aub_os_rr = os_rr[os_rr['PitcherTeam']=='AUB_TIG']
print(aub_os_rr.shape)

In [ ]:
# Get unique values
unique_values = os_rr['Pitcher'].unique()
aub_unique_values = aub_os_rr['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: os_rr[os_rr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: os_rr[os_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_os_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(os_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} offspeed rr stuff+: {np.mean(stuff * 100)}")

    


In [ ]:
#Stuff+ for selected Auburn pitchers offspeed in RHP v RHB split
for pitcher in aub_unique_values:
    aub_rr_os_stuff(pitcher)

Righty vs. Lefty Split

In [ ]:
os_rl_X = os_rl[preds]
os_rl_y = os_rl['whiff']

In [ ]:
#Train Model: RHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(os_rl_X, os_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_os_rl = os_rl[os_rl['PitcherTeam']=='AUB_TIG']
print(aub_os_rl.shape)

In [ ]:
# Get unique values
unique_values = os_rl['Pitcher'].unique()
aub_unique_values = aub_os_rl['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: os_rl[os_rl['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: os_rl[os_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_os_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(os_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} offspeed rl stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers offspeed in RHP v LHB split
for pitcher in aub_unique_values:
    aub_rl_os_stuff(pitcher)

Lefty vs. Righty Splits

In [ ]:
os_lr_X = os_lr[preds]
os_lr_y = os_lr['whiff']

In [ ]:
#Train Model: LHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(os_lr_X, os_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_os_lr = os_lr[os_lr['PitcherTeam']=='AUB_TIG']
print(aub_os_lr.shape)

In [ ]:
# Get unique values
unique_values = os_lr['Pitcher'].unique()
aub_unique_values = aub_os_lr['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: os_lr[os_lr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: os_lr[os_lr['Pitcher'] == value] for value in aub_unique_values}


In [ ]:
def aub_lr_os_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(os_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} offspeed lr stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers offspeed in LHP v RHB split
for pitcher in aub_unique_values:
    aub_lr_os_stuff(pitcher)

Lefty vs. Lefty Splits

In [ ]:
os_ll_X = os_ll[preds]
os_ll_y = os_ll['whiff']

In [ ]:
#Train Model: LHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(os_ll_X, os_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_os_ll = os_ll[os_ll['PitcherTeam']=='AUB_TIG']
print(aub_os_ll.shape)

In [ ]:
# Get unique values
unique_values = os_ll['Pitcher'].unique()
aub_unique_values = aub_os_ll['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: os_ll[os_ll['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: os_ll[os_ll['Pitcher'] == value] for value in aub_unique_values}


In [ ]:
def aub_ll_os_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(os_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} offspeed ll stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers offspeed in LHP v LHB split
for pitcher in aub_unique_values:
    aub_ll_os_stuff(pitcher)

In [ ]:
aub_subsets['Crotchfelt, Zach']